In [9]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import sqlite_table_one_to_many_datastore as sqlotmds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from transformer import transformers as trs
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
sqlite_db_path='ds_data\siftid_sift_imgid'
sqlite_table_name='siftid_sift_imgid'

We can interpret table as One-to-One(id, itemid) or as One-to-Many(id, itemid, foreignid).
If we want to us it like One-to-One, use SQLiteTableDataStore. Here item = one sift.
If we want to us it like One-to-Many, use SQLiteTableOneToManyDataStore. In this case items would be grouped by foreignid and aggregated into ndarray. Here item = ndarray of sifts, e.g. ndarray with shape (312, 128)

In [11]:
ds_in = fsds.FileSystemDirectoryDataStore(dir_path=img_dir_path)
ds_out = sqlotmds.SQLiteTableOneToManyDataStore(sqlite_db_path, sqlite_table_name)
print(ds_in.get_count())

999


In [12]:
transformers_=[trs.BytesToNdarray(), trs.NdarrayToOpencvMatrix(), trs.OpencvMatrixToSiftsSet()]
steps.transform_step(ds_in, transformers_, ds_out)

In [13]:
with ds_out:
    print(ds_out.get_count())

999


In [14]:
with ds_out:
    ds_out.get_items_sorted_by_ids()

In [15]:
with ds_out:
    items=ds_out.get_items_sorted_by_ids()
    for i in range(3):
        print(next(items).shape)

(379, 128)
(407, 128)
(395, 128)


In [ ]:
import os
os.remove(sqlite_db_path)